
Relationship Prediction
===========================================

Notebook for training and inference of element connectivity prediction using GNNs

In [ ]:
%matplotlib inline

In [ ]:
import dgl
from dgl.data import DGLDataset
import torch
import time
import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import math
from pathlib import Path
import scipy.sparse as sp
#from google.colab import drive
import pickle
from tqdm.notebook import tqdm

from src.graph import get_edges_from_node_info_np, IndustrialFacilityDataset
from src.geometry import sq_dist_vect

### Load graph dataset




In [ ]:
# paths

#model_path = '/content/drive/MyDrive/graph/'
model_path = 'gnn_params/'
model_name = "model_4sage_3mlp.pth"
pred_name = "pred_4sage_3mlp.pth"
data_path = "/mnt/c/data/3D_CAD/"


In [ ]:
types = ['FLANGE', 'ELBOW', 'TEE', 'TUBE', 'BEND']
np.random.seed(42)
test_mode = False
use_params = True
create_pseudo_graph = True
pseudo_graph_dist_thresh = 0.5
negative_sampling_ratio = 1
device = torch.device("cuda:0")


In [ ]:
if not test_mode:
    path = Path('output/west_ref/')
    sitename = 'west'
    dataset = IndustrialFacilityDataset(data_path, "westdeckbox_ref", use_params, path, sitename)
else:
    path = Path('output/east_ref/')
    sitename = 'east'
    dataset = IndustrialFacilityDataset(data_path, "eastdeckbox", use_params, path, sitename)
g = dataset[0]
g = g.to(device)
print(g)

#### pseudo graph generation

In [ ]:
# generate a pseudo graph whose edges depict nodes that are within a distance threshold
def generate_pseudo_graph(nodes, dist_thresh=0.5):
    print(len(nodes)**2)
    nodes_cpu = nodes.cpu().numpy()
    pseudo_edges = []
    for i, n0 in enumerate(tqdm(nodes_cpu)):
        for j, n1 in enumerate(nodes_cpu):
            
            if i >=j:
                continue
                
            #rough distance check using two edges
            bb0 = get_edges_from_node_info_np(n0)
            bb1 = get_edges_from_node_info_np(n1)
            if ((sq_dist_vect(bb0[0], bb1[0]) < dist_thresh) or
                (sq_dist_vect(bb0[0], bb1[1]) < dist_thresh) or
                (sq_dist_vect(bb0[1], bb1[0]) < dist_thresh) or
                (sq_dist_vect(bb0[1], bb1[1]) < dist_thresh)):
                
                pseudo_edges.append((i,j))

    return pseudo_edges


In [ ]:
if create_pseudo_graph:
    pseudo_edges = generate_pseudo_graph(g.ndata['feat'], pseudo_graph_dist_thresh)
    
    with open(model_path + '/pseudo_graph_' + sitename + str(pseudo_graph_dist_thresh) + '.pkl', 'wb') as f:
        pickle.dump(pseudo_edges, f)

else:
    with open(model_path + '/pseudo_graph_' + sitename + str(pseudo_graph_dist_thresh) + '.pkl', 'rb') as f:
       pseudo_edges = pickle.load(f)

In [ ]:
if test_mode:
    # remove real edges and add pseudo edges for testing
    # TODO: compare the pseudo edges vs. real edges
    print(g.number_of_edges(), len(pseudo_edges))
    eids = np.arange(g.number_of_edges())
    g_test = dgl.remove_edges(g, eids)
    
    pseudo_u = [e[0] for e in pseudo_edges] + [e[1] for e in pseudo_edges]
    pseudo_v = [e[1] for e in pseudo_edges] + [e[0] for e in pseudo_edges]
    g_test.add_edges(np.array(pseudo_u), np.array(pseudo_v))
    print(g_test.number_of_edges())
    
    
    

#### debugging element ids

In [ ]:
# node ids don't seem to match - probably the graph dataset was extracted from a different version of ifc file with different element ids
# first, check ids in the merged file - compare with the ids from the param data
# also check ids in the merged file with node dataset

# import ifcopenshell
# from ifcopenshell.util.selector import Selector

# ifc = ifcopenshell.open(data_path +"merged.ifc")

In [ ]:
#ifc_tee = ifcopenshell.open(data_path +"deckboxtee_ref.ifc")

In [ ]:
# element_type = 'IFCPIPEFITTING'
# #element_type = 'IFCPIPESEGMENT'
# selector = Selector()
# elements = selector.parse(ifc, '.' + element_type)
# print(len(elements))
# ids= []
# for e in elements:
#     ids.append(e.id())
# print(len(ids), ids[:10])

In [ ]:
# with open(data_path + 'nodes_westdeckbox_ref.pkl', 'rb') as f:
#     node_info = pickle.load(f)
#     nodes = node_info[0]
    

In [ ]:
# ids_tee = [n[4] for n in nodes if n[0]==1]
# #ids_tee = [n[4] for n in nodes]
# print(len(ids_tee))
# # node_ids = [n[4] for n in nodes if n[0]==3 or n[0]==4]
# # print(len(node_ids))

In [ ]:

# elements_tee = selector.parse(ifc_tee, '.' + element_type)
# print(len(elements_tee))
# ids_tee= []
# for e in elements_tee:
#     ids_tee.append(e.id())
# print(len(ids_tee), ids_tee[:10], max(ids_tee), max(ids))

In [ ]:
# matches, non_matches = [], []

# for id1 in tqdm(ids_tee):
#     found = False
#     for i, id2 in enumerate(ids):
#         if id1 == id2:
#             matches.append((id1, i))
#             found = True
#             break
#     if not found:
#         non_matches.append(id1)
               
# print(len(matches), len(non_matches))


Prepare training and testing sets
---------------------------------

This cell randomly picks 10% of the edges for positive examples in
the test set, and leaves the rest for the training set. It then samples
the same number of edges for negative examples in both sets.

Ignore for evaluation.



In [ ]:
# Split edge set for training and testing
if not test_mode:
    u, v = g.edges()

    eids = np.arange(g.number_of_edges())
    eids = np.random.permutation(eids)
    test_size = int(len(eids) * 0.1)
    train_size = g.number_of_edges() - test_size
    test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
    train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

    # Find all negative edges and split them for training and testing
    #print(u.numpy().shape, v.numpy().shape)

    adj = sp.coo_matrix((np.ones(len(u)), (u.cpu().numpy(), v.cpu().numpy())), 
                        shape=(g.number_of_nodes(), g.number_of_nodes()))
    #print(adj.shape, adj.todense().shape, np.eye(g.number_of_nodes()).shape)
    adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
    neg_u, neg_v = np.where(adj_neg != 0)


In [ ]:
# dynamically sample negative edges to create negative graph
def construct_negative_graph(g, total_train_neg_eids, train_size):
#     t1 = time.perf_counter()
    train_neg_eids_eids = np.random.choice(len(total_train_neg_eids), train_size)
    train_neg_eids = total_train_neg_eids[train_neg_eids_eids]
    train_neg_u = neg_u[train_neg_eids] 
    train_neg_v = neg_v[train_neg_eids]
    train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())
    train_neg_g = train_neg_g.to(device)

#     t2 = time.perf_counter()
#     print("time", t2-t1)
#     print(train_neg_eids[:10])
    return train_neg_g


In [ ]:
# sample a test set, and isolate all remaining negative edges for sampling later
if not test_mode:
    test_neg_eids = np.random.choice(len(neg_u), test_size*negative_sampling_ratio)
    test_neg_eids = np.sort(test_neg_eids)
    all_indices = np.arange(len(neg_u))
    total_train_neg_eids = np.delete(all_indices, test_neg_eids)
    print(len(test_neg_eids), len(total_train_neg_eids))

#     total_train_neg_eids = []
#     k = 0
#     for i in tqdm(range(len(neg_u))):
#         if k==len(test_neg_eids):
#             total_train_neg_eids.append(i)
#             continue
#         if i != test_neg_eids[k]:
#             total_train_neg_eids.append(i)
#         else:
#             k+=1
#     print(len(test_neg_eids) + len(total_train_neg_eids) - len(neg_u))

    test_neg_u, test_neg_v = neg_u[test_neg_eids], neg_v[test_neg_eids]
    total_train_neg_eids = np.array(total_train_neg_eids)


In [ ]:
# remove edges of testset for training
if not test_mode:
    train_g = dgl.remove_edges(g, eids[:test_size])

In [ ]:
# construct the positive graph and the negative graph for the training set and the test set respectively.
if not test_mode:
    train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
    #train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

    test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
    test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())
    test_pos_g = test_pos_g.to(device)
    test_neg_g = test_neg_g.to(device)

GraphSAGE model
-------------------





In [ ]:
impute_missing = False
use_custom_feats = True

In [ ]:
# extact radius, direction and position of edge from feature tensor
def get_rdp_tensor(feat, e, feat_size):
    r = feat[:, feat_size+1+e]
    p = torch.column_stack([feat[:, feat_size+4+e*3], 
                            feat[:, feat_size+5+e*3], 
                            feat[:, feat_size+6+e*3]])
    d = torch.column_stack([feat[:, feat_size+13+e*3], 
                            feat[:, feat_size+14+e*3], 
                            feat[:, feat_size+15+e*3]])
    return r, d, p


# get angle deviation between two pipe centerlines
def get_centerline_deviation_tensor(d1, d2):
    dev = torch.acos( torch.sum(d1*d2, dim=-1))
    dev = torch.where(dev > np.pi/2, np.pi - dev, dev)
    return dev


# get euclidean distance between two points
def get_point_dist_tensor(p1, p2):
    return(torch.sqrt(torch.sum((p1-p2).pow(2), -1)))


# replace edge features with zeros where radius is zero
def replace_for_zero_r_tensor(inp, src_r, tgt_r):
    return torch.where(torch.logical_or(src_r == 0, tgt_r == 0),
                       torch.zeros(inp.shape).to(device),
                       inp)


#TODO: intergrate dist to intersection as edge feature
def get_distance_to_intersection_tensor(p1, p2, d1, d2):
    centerline_connecting_line = F.normalize(torch.cross(d1, d2))
    center_connecting_line = p2 - p1
    centerline_distance = (torch.abs(torch.sum(centerline_connecting_line*center_connecting_line, 
                                               dim=-1)) / 
                           torch.linalg.vector_norm(centerline_connecting_line, dim=1))
    sq_mag_ccl = torch.sum(centerline_connecting_line*centerline_connecting_line,
                           dim=-1)
    t1 = torch.sum(torch.cross(d2, centerline_connecting_line)*center_connecting_line, 
                   dim=-1) / sq_mag_ccl
    t2 = torch.sum(torch.cross(d1, centerline_connecting_line)*center_connecting_line, 
                   dim=-1) / sq_mag_ccl
    
    #print(t1.shape, t2.shape)
    return t1, t2


def get_distance_to_intersection(a, b):
    centerline_connecting_line = np.cross(a[3], b[3])
    center_connecting_line = b[1] - a[1]
    centerline_distance = (abs(np.dot(centerline_connecting_line, 
                                          center_connecting_line)) / 
                           vector_mag(centerline_connecting_line))
    sq_mag_ccl = np.dot(centerline_connecting_line, 
                        centerline_connecting_line)
    t1 = np.dot(np.cross(b[3], centerline_connecting_line),
                center_connecting_line) / sq_mag_ccl
    t2 = np.dot(np.cross(a[3], centerline_connecting_line),
                center_connecting_line) / sq_mag_ccl
    
    return t1, t2

In [ ]:
# calculate custom edge features
def get_edge_features(src_nodes, tgt_nodes):
    feature_columns = []

    # iterate through edges of first element
    for e1 in range(3):
        src_r, src_d, src_p = get_rdp_tensor(src_nodes, e1, 0)
        
        # iterate through edges of second element
        for e2 in range(3):
            tgt_r, tgt_d, tgt_p = get_rdp_tensor(tgt_nodes, e2, 0)

            # compute edge features
            r_ratio = src_r / tgt_r
            deviation = get_centerline_deviation_tensor(src_d, tgt_d)
            point_dist = get_point_dist_tensor(src_p, tgt_p)
            t1, t2 = get_distance_to_intersection_tensor(src_p, tgt_p, src_d, tgt_d)

            # remove edge features when one edge has zero radius
            r_ratio = replace_for_zero_r_tensor(r_ratio, src_r, tgt_r)
            deviation = replace_for_zero_r_tensor(deviation, src_r, tgt_r)
            point_dist = replace_for_zero_r_tensor(point_dist, src_r, tgt_r)
            t1 = replace_for_zero_r_tensor(t1, src_r, tgt_r)
            t2 = replace_for_zero_r_tensor(t2, src_r, tgt_r)

            feature_columns.append(r_ratio)
            feature_columns.append(deviation)
            feature_columns.append(point_dist)
            #feature_columns.append(t1)
            #feature_columns.append(t2)
            
            
    feature_columns.append(src_nodes[:, 0])
    feature_columns.append(tgt_nodes[:, 0])
            

    feature_columns_tensor = torch.column_stack(feature_columns)
    #print(feature_columns_tensor.shape)
    return feature_columns_tensor


In [ ]:
from dgl.nn import SAGEConv

# ----------- 2. create model -------------- #
# build a 3-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')
        self.conv3 = SAGEConv(h_feats, h_feats, 'mean')
#         self.conv4 = SAGEConv(h_feats, h_feats, 'mean')
#         self.conv5 = SAGEConv(h_feats, h_feats, 'mean')
#         self.conv6 = SAGEConv(h_feats, h_feats, 'mean')
#         self.conv7 = SAGEConv(h_feats, h_feats, 'mean')
#         self.conv8 = SAGEConv(h_feats, h_feats, 'mean')
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        h = F.relu(h)
        h = self.conv3(g, h)
#         h = F.relu(h)
#         h = self.conv4(g, h)
#         h = F.relu(h)
#         h = self.conv5(g, h)
#         h = F.relu(h)
#         h = self.conv6(g, h)
#         h = F.relu(h)
#         h = self.conv7(g, h)
#         h = F.relu(h)
#         h = self.conv8(g, h)
        return h

In [ ]:
# compute edge features using dot product

import dgl.function as fn

class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata['score'][:, 0]

In [ ]:
# ALTERNATIVE: compute edge features using an MLP

class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.h_feats = h_feats
        custom_feats = 0
        if use_custom_feats:
            custom_feats = 29

#         self.W1 = nn.Linear(h_feats * 2+ custom_feats, h_feats)
#         self.W2 = nn.Linear(h_feats, 1)

#         self.W1 = nn.Linear(h_feats * 2 + custom_feats, h_feats)
#         self.W2 = nn.Linear(h_feats, int(h_feats/2))
#         self.W3 = nn.Linear(int(h_feats/2), 1)

#         self.W1 = nn.Linear(h_feats * 2 + custom_feats, h_feats)
#         self.W2 = nn.Linear(h_feats, int(h_feats/2))
#         self.W3 = nn.Linear(int(h_feats/2), 4)
#         self.W4 = nn.Linear(4, 1)

        self.W1 = nn.Linear(h_feats * 2 + custom_feats, h_feats*2)
        self.W2 = nn.Linear(h_feats*2, h_feats)
        self.W3 = nn.Linear(h_feats, int(h_feats/2))
        self.W4 = nn.Linear(int(h_feats/2), 4)
        self.W5 = nn.Linear(4, 1)
        
#         self.W1 = nn.Linear(h_feats * 2 + custom_feats, h_feats*4)
#         self.W2 = nn.Linear(h_feats*4, h_feats*2)
#         self.W3 = nn.Linear(h_feats*2, h_feats)
#         self.W4 = nn.Linear(4, 1)

    def apply_edges(self, edges):
        """
        Computes a scalar score for each edge of the given graph.

        Parameters
        ----------
        edges :
            Has three members ``src``, ``dst`` and ``data``, each of
            which is a dictionary representing the features of the
            source nodes, the destination nodes, and the edges
            themselves.

        Returns
        -------
        dict
            A dictionary of new edge features.
        """
        
        # compute custom edge features
        if use_custom_feats:
            edge_features = get_edge_features(edges.src['h'][:,self.h_feats:], 
                                              edges.dst['h'][:,self.h_feats:])

            h = torch.cat([edges.src['h'][:,:self.h_feats], 
                           edges.dst['h'][:,:self.h_feats], 
                           edge_features], 
                      1)
        else:
            h = torch.cat([edges.src['h'], edges.dst['h']], 1)
            
        #print("in", edges.src['h'].shape, "edge", edge_features.shape, "out", h.shape)

        #return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}
        #return {'score': self.W3(F.relu(self.W2(F.relu(self.W1(h))))).squeeze(1)}
        #return {'score': self.W4(F.relu(self.W3(F.relu(self.W2(F.relu(self.W1(h))))))).squeeze(1)}
        return {'score': self.W5(F.relu(self.W4(F.relu(self.W3(F.relu(self.W2(F.relu(self.W1(h))))))))).squeeze(1)}

        #return {'score': self.W1(h).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']

Training setup
-------------



The loss function is binary cross entropy loss.

\begin{align}\mathcal{L} = -\sum_{u\sim v\in \mathcal{D}}\left( y_{u\sim v}\log(\hat{y}_{u\sim v}) + (1-y_{u\sim v})\log(1-\hat{y}_{u\sim v})) \right)\end{align}

The evaluation metric  is AUC.




In [ ]:
print(g.ndata['feat'].shape[1])

In [ ]:
from sklearn.metrics import roc_auc_score

feat_size = 16
custom_edge_feature_size = 22

if test_mode:
  model = GraphSAGE(g.ndata['feat'].shape[1], feat_size)
else:
  model = GraphSAGE(train_g.ndata['feat'].shape[1], feat_size)

model = model.to(torch.double)
model = model.to(device)

# You can replace DotPredictor with MLPPredictor.
pred = MLPPredictor(feat_size)
# pred = DotPredictor()
pred = pred.to(torch.double)
pred = pred.to(device)


In [ ]:
def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    #print(scores)
    labels = torch.cat([torch.ones(pos_score.shape[0]).to(device), 
                        torch.zeros(neg_score.shape[0]).to(device)])
    #w = torch.as_tensor([0.01 for i in range(len(labels))])
    #return F.binary_cross_entropy_with_logits(scores, labels, w)
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).cpu().numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)

In [ ]:
from numpy.lib.function_base import average
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pos_score, neg_score, threshold=0.8):
    # for x in torch.cat([pos_score, neg_score]):
    #   print(x)

    scores = torch.sigmoid(torch.cat([pos_score, neg_score])).cpu().numpy()
    #scores = np.rint(scores).astype(int)
    scores[scores > threshold] = 1
    scores[scores != 1] = 0
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    #print(np.average(scores), scores.shape, labels.shape)
    return (accuracy_score(labels, scores), precision_score(labels, scores), 
            recall_score(labels, scores), f1_score(labels, scores))


In [ ]:
def impute_missing_with_mean(feat):
    #print(feat[:20])
    r_key = 3
    fields = [10, 11, 12, 19, 20, 21, 3]
    for f in fields:
        #print(torch.count_nonzero(feat[:,r_key]))
        mean = torch.sum(feat[:,f]) / torch.count_nonzero(feat[:,r_key])
        mean_tensor = torch.full(feat[:,f].shape, mean).to(device)
        #print(mean, mean_tensor.shape)
        feat_copy = torch.clone(feat)
        feat_copy[:,f] = torch.where(feat_copy[:,r_key] == 0, mean_tensor, feat_copy[:,f])
    return feat

In [ ]:
if impute_missing:
    if test_mode:
        imputed_feat_g = impute_missing_with_mean(g.ndata['feat'])
    else:
        imputed_feat = impute_missing_with_mean(train_g.ndata['feat'])
            

### Evaluation 
load pre-trained model and run on entire dataset in batches (due to the large size of negative dataset)

In [ ]:
if test_mode:

    # load model
    model.load_state_dict(torch.load(model_path + model_name))
    pred.load_state_dict(torch.load(model_path + pred_name))

In [ ]:
# validation set
if test_mode:
  u, v = g.edges()
  eids = np.arange(g.number_of_edges())
  
  adj = sp.coo_matrix((np.ones(len(u)), (u.cpu().numpy(), v.cpu().numpy())), 
                      shape=(g.number_of_nodes(), g.number_of_nodes()))
  adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
  neg_u_full, neg_v_full = np.where(adj_neg != 0)
  print(len(neg_u_full))
  #neg_eids = np.random.choice(len(neg_u), g.number_of_edges())


In [ ]:
if test_mode:    
    with open(model_path + 'eval/pos_edges_test.pkl', 'wb') as f:
                pickle.dump([u,v], f)
    with open(model_path + 'eval/neg_edges_test.pkl', 'wb') as f:
                pickle.dump([neg_u_full,neg_v_full], f)
    print(len(u), len(neg_u_full))

In [ ]:
if test_mode:
    with torch.no_grad():
       
        if impute_missing:
            h = model(g_test, imputed_feat_g)
        else:
            h = model(g_test, g.ndata['feat'])

        if use_custom_feats:
            h = torch.cat([h, g.ndata['feat'][:,:custom_edge_feature_size]], 1)
        
        # evaluate positive examples
        val_pos_g = dgl.graph((u, v), num_nodes=g.number_of_nodes())
        pos_score = pred(val_pos_g, h).cpu().numpy()
        
        with open(model_path + '/eval/pos_score_test.pkl', 'wb') as f:
            pickle.dump(pos_score, f)

        # evaluate negative examples
        sample_size = 2000000
        limit = math.ceil(len(neg_u_full)/sample_size)
        #print(len(neg_u_full), limit)
        neg_scores = []
        
        for i in tqdm(range(limit)):
            if i == limit-1:
                neg_u, neg_v = neg_u_full[i*sample_size:], neg_v_full[i*sample_size:]
            else:
                neg_u = neg_u_full[i*sample_size:(i+1)*sample_size]
                neg_v = neg_v_full[i*sample_size:(i+1)*sample_size]
            #print(i, len(neg_u), neg_u[0],  neg_v[0])

            val_neg_g = dgl.graph((neg_u, neg_v), num_nodes=g.number_of_nodes())
            val_neg_g = val_neg_g.to(device)
            neg_scores.append(pred(val_neg_g, h).cpu().numpy())
        
        neg_score = np.concatenate(neg_scores)
        with open(model_path + '/eval/neg_score_test.pkl', 'wb') as f:
            pickle.dump(neg_score, f)
#             print('AUC', compute_auc(pos_score, neg_score))

In [ ]:
if test_mode:

    with open(model_path + '/eval/pos_score_test.pkl', 'rb') as f:
        pos_score = pickle.load(f)

    with open(model_path + '/eval/neg_score_test.pkl', 'rb') as f:
        neg_score = pickle.load(f)

In [ ]:
if test_mode:
    print(len(pos_score), len(neg_score), sum(pos_score)/len(pos_score), sum(neg_score)/len(neg_score))

In [ ]:
if test_mode:
    metrics = compute_metrics(torch.from_numpy(pos_score), torch.from_numpy(neg_score))
    print('accuracy', metrics[0])
    print('precision', metrics[1])
    print('recall', metrics[2])
    print('f1_score', metrics[3])

### train / validate

In [ ]:
# ----------- 3. set up loss and optimizer -------------- #
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.001)

# ----------- 4. training -------------------------------- #
all_logits = []
accuracies = []
losses = []


for e in tqdm(range(1500)):
    # forward
    #print(len(c))
    train_neg_g = construct_negative_graph(g, total_train_neg_eids, train_size*negative_sampling_ratio)
    #print(train_g.ndata['feat'].device, train_g.device)
    if impute_missing:
        h = model(train_g, imputed_feat)
    else:
        h = model(train_g, train_g.ndata['feat'])
    
    # concatanate original node features for custom edge feature computation
    if use_custom_feats:
        h = torch.cat([h, train_g.ndata['feat'][:,:custom_edge_feature_size]], 1)

    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    loss = compute_loss(pos_score, neg_score)
    
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    highest_f1 = 0
    
    if e % 5 == 0:
        with torch.no_grad():
          pos_score_test = pred(test_pos_g, h)
          neg_score_test = pred(test_neg_g, h)
          auc =  compute_auc(pos_score_test, neg_score_test)
          metrics = compute_metrics(pos_score_test, neg_score_test)
          test_loss = compute_loss(pos_score_test, neg_score_test)
          res_sring = 'epoch {}, training loss: {:.3f}, test loss: {:.3f}, auc: {:.3f}, f1: {:.3f}, pr: {:.3f}, re{:.3f}'
          print(res_sring.format(e, loss, test_loss, auc, float(metrics[3]), 
                                 float(metrics[1]), float(metrics[2])))
        
          if metrics[3] > highest_f1:
            torch.save(model.state_dict(), model_path + model_name)
            torch.save(pred.state_dict(), model_path + pred_name)
          # metrics = compute_metrics(pos_score, neg_score)
          accuracies.append(auc)
          losses.append(loss.item())


# ----------- 5. check results ------------------------ #
print("max AUC", max(accuracies))
with torch.no_grad():
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    print('AUC', compute_auc(pos_score, neg_score))
    
# for i in range(len(losses)):
#     print(i*5, losses[i], accuracies[i])
#     print('AUC', compute_auc(pos_score, neg_score))



In [ ]:
model.load_state_dict(torch.load(model_path + model_name))
pred.load_state_dict(torch.load(model_path + pred_name))

In [ ]:

with torch.no_grad():
    #h = model(train_g, train_g.ndata['feat'])
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    metrics = compute_metrics(pos_score, neg_score)
    print('accuracy', metrics[0])
    print('precision', metrics[1])
    print('recall', metrics[2])
    print('f1_score', metrics[3])
    print('auc', compute_auc(pos_score, neg_score))

In [ ]:
print(len(test_pos_u), len(test_neg_u))

In [ ]:
# calculate separate metrics based on element classes
# print(g.ndata["feat"][test_pos_u[0]])
# print(np.unique(g.ndata["feat"].numpy()[:,0]))

type_pairs = [(0., 0.), (0., 1.), (0., 2.), (0., 3.), (0., 4.), 
              (1., 1.), (1., 2.), (1., 3.), (1., 4.), (2., 2.), 
              (2., 3.), (2., 4.), (3., 3.), (3., 4.), (4., 4.)]

def split_graph(u, v, node_features, type_pairs):
    subgraph_edges = {pair: ([], []) for pair in type_pairs}
    subgraphs = {}
    
    for i in range(len(u)):
        u_type = node_features[u[i], 0]
        v_type = node_features[v[i], 0]
        
        edge_type = (u_type, v_type)
        if edge_type in subgraph_edges:
            subgraph_edges[edge_type][0].append(u[i])
            subgraph_edges[edge_type][1].append(v[i])

    
    for pair, (sub_u, sub_v) in subgraph_edges.items():
        pos_g = dgl.graph((sub_u, sub_v), num_nodes=g.number_of_nodes())
        #test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())
        subgraphs[pair] = pos_g.to(device)
        
    return subgraphs

pos_subgraphs = split_graph(test_pos_u, test_pos_v, g.ndata["feat"].cpu().numpy(), type_pairs)
neg_subgraphs = split_graph(test_neg_u, test_neg_v, g.ndata["feat"].cpu().numpy(), type_pairs)
#print(neg_subgraphs)


for pair, pos_sub_g in pos_subgraphs.items():
    print((types[int(pair[0])], types[int(pair[1])]))
    print("positive edges", pos_sub_g.num_edges())
    print("negative edges", neg_subgraphs[pair].num_edges())
#     print(f"Subgraph for type pair {pair}:")
#     print(f"u: {sub_u}")
#     print(f"v: {sub_v}\n")

    with torch.no_grad():
        pos_score = pred(pos_sub_g, h)
        neg_score = pred(neg_subgraphs[pair], h)
        try:
            metrics = compute_metrics(pos_score, neg_score)
            print('accuracy', metrics[0])
            print('precision', metrics[1])
            print('recall', metrics[2])
            print('f1_score', metrics[3])
            print('auc', compute_auc(pos_score, neg_score))
        except:
            print("metric error")


In [ ]:
# save model
torch.save(model.state_dict(), model_path + model_name)
torch.save(pred.state_dict(), model_path + pred_name)